In [68]:
!ls *.sh

MIROC5_historical_future_cloudVegET.sh


In [1]:
# Previously for CCSM4 model, which does not have max/min relative humidity needed to find vapor pressure deficit
#with open('macav2metdata-allnetcdf_CONUS.sh') as fo:
#    lines = fo.readlines()

with open('MIROC5_historical_future_cloudVegET.sh') as fo:
    lines = fo.readlines() 

In [2]:
# explore some of the output
print(lines)

words = lines[1].split()
for w in words:
    if 'http' in w:
        print(w)
        http_line = w
# print(http_line)

# http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
# print(http_root)

['#!/bin/bash \n', 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc  \n', 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily.nc  \n', 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily.nc  \n', 'wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc" -O macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc  \n', 'wget -nc -c -nd "http://thre

In [3]:
import pandas as pd
import re
def grepfxn(pattern,files):
    # Purpose: this function 
    precip_asc_df2=pd.DataFrame(files)
    idx_asc_in2 = []
    for i, precipfile in precip_asc_df2.iterrows():
        val = re.findall(pattern, precipfile.iloc[0])
        if len(val) > 0:
            idx_asc_in2.append(i)
            
    ascfileround2 = [files[i] for i in idx_asc_in2]
    return ascfileround2

In [6]:
import pandas as pd  
import os

with open('MIROC5_historical_future_cloudVegET.sh') as fo:
    lines = fo.readlines()

# Extract name of netCDF from shell script
outs=[]
for line in lines:
    if '-O' in line:
        out=line.split('-O')[1][1:]
        out = out.rstrip()
        out = out.split('.nc')[0]
        print(out)
        outs.append(out)



macav2metdata_pr_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1955_1959_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1960_1964_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1970_1974_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1975_1979_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1980_1984_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1985_1989_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1990_1994_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_1995_1999_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_2000_2004_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_historical_2005_2005_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2011_2015_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2016_2020_CONUS_daily
macav2metdata_pr_MIROC5_r1i1p1_rcp45_2021_2025_CONUS_da

In [ ]:
# THIS IS FOR THE ORIGINAL DOWNLOAD THAT DOWNLOADED MOST FILES SUCCESSFULLY

import pandas as pd  
import os

with open('MIROC5_historical_future_cloudVegET.sh') as fo:
    lines = fo.readlines()

# Extract name of netCDF from shell script
outs=[]
for line in lines:
    if '-O' in line:
        out=line.split('-O')[1][1:]
        out = out.rstrip()
        out = out.split('.nc')[0]
        print(out)
        outs.append(out)

# break down the name of each file into each metadata component
dfis=[]
for out in outs:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)
    
    

In [ ]:


# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

# Get climate variable names - will be needed later when moving into the cloud
# print(df['measure'].values)

home = os.getenv('HOME')
wget = 'wget -nc -c -nd ' 

print(df)

In [4]:
import boto3
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)


In [ ]:
# This was just to look at that the code that is within the for loop below (9/10)

pr_df = df[df['measure'] == 'pr']

# extract all of the rows that have 'pr' or precipitation within the netcdf file name
# pr_df= pr_df[pr_df['start_year'] == '2006']
# pr_lines = lines[13]
print(pr_df)
# print(pr_lines)
# print(pr_df.iloc[12])
# print(range(0,len(pr_df.index)))
# print(pr_df.loc)

myd = dict(pr_df)
myl = list(pr_df) # get each precipitation row
print(myl)

myd = dict(pr_df)
myl = list(pr_df.iloc[1]) # get each precipitation row
http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
crafted_file_name = '_'.join(myl) + '.nc'
wget = 'wget -nc -c -nd ' 

to_dir= home + '/junkbox/' + myd['measure']

wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
wget_cmd2 = wget + wget_cmd.iloc[0]
print(wget_cmd2)

try:
    os.mkdir(to_dir);
except:
    pass

os.system(wget_cmd2)

bucket = 'dev-et-data'

local_file = to_dir + '/' + crafted_file_name
print(local_file)


lines[1]
pr_df = df[df['measure'] == 'pr']
pr_lines = lines[1:32]
myl = list(pr_df.loc[30])
# myl = list(pr_df)
print(myl)

to_dir= home + '/junkbox/' + myd['measure']
print(to_dir)
crafted_file_name = '_'.join(myl) + '.nc'
bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
print(bucket_filepath)

In [ ]:
# THIS IS THE ACTUAL PART OF THE CODE THAT 1) DOWNLOADS .NC FILES TO MINI-PANGEO 2) AND PUSHES THEM TO THE RAW FOLDER ON THE
# MINI-PANGEO. NOTE - ADDITIONAL CODE IS NEEDED AFTER THIS POINT TO MOVE THE INPUTS FROM RAW FOLDER TO 'IN' FOLDER. LOOKING
# BACK, WHAT I SHOULD HAVE DONE IS JUST PUT THEM IN THE 'IN' FOLDER IN THE FIRST PLACE. IF I HAVE TIME, I WILL CHANGE
# THE CODE BELOW TO REFLECT THIS (Gregory Rouze, 9/10/2020)

# Example of command line move of input data: aws s3 mv s3://dev-et-data/raw/tasmin s3://dev-et-data/in/DelawareRiverBasin/Temp/Climatology/Tmin/ --recursive

In [ ]:
pr_df = df[df['measure'] == 'pr'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
pr_lines = lines[1:32] # 1 to 31 are the rows associated with precipitation netCDF files

for i in range(0,len(pr_df.index)):
    words = pr_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(pr_df.loc[i+1])
    myl = list(pr_df.loc[i+1]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd.iloc[0]
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

# Everything below this point is the same code, except for different variables, which requires different indexing values
    

In [ ]:
rhsmax_df = df[df['measure'] == 'rhsmax']
rhsmax_lines = lines[32:63]
myl = list(rhsmax_df.loc[31])
print(myl)

In [ ]:
rhsmax_df = df[df['measure'] == 'rhsmax']
rhsmax_lines = lines[32:63]

for i in range(0,len(rhsmax_df.index)):
    words = rhsmax_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rhsmax_df.loc[i+31])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rhsmax_df.loc[i+31])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
rhsmin_df = df[df['measure'] == 'rhsmin']
rhsmin_lines = lines[63:94]

for i in range(0,len(rhsmin_df.index)):
    words = rhsmin_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rhsmin_df.loc[i+62])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rhsmin_df.loc[i+62])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
# rsds_df = df[df['measure'] == 'rsds']
# print(rsds_df)
# rsds_lines = lines[32:63]
# print(rsds_lines)
# len(rsds_df.index)
# rsds_df.loc[31]

rsds_df = df[df['measure'] == 'rsds']
rsds_lines = lines[94:125]

for i in range(0,len(rsds_df.index)):
    words = rsds_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(rsds_df.loc[i+93]) # +31 is done to identify the row numbers from the beginning of the dataframe (i.e. doesn't start at 0)
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(rsds_df.loc[i+93])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
tasmin_df = df[df['measure'] == 'tasmin']
tasmin_lines = lines[125:156]

for i in range(0,len(tasmin_df.index)):
    words = tasmin_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(tasmin_df.loc[i+124])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(tasmin_df.loc[i+124])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
tasmax_df = df[df['measure'] == 'tasmax']
tasmax_lines = lines[156:187]

for i in range(0,len(tasmax_df.index)):
    words = tasmax_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(tasmax_df.loc[i+155])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(tasmax_df.loc[i+155])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
uas_df = df[df['measure'] == 'uas']
uas_lines = lines[187:218]

for i in range(0,2):
    words = uas_lines[i].split()
    for w in words:
        if 'http' in w:
#             print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(uas_df.loc[i+186])
    
    to_dir= home + '/junkbox/' + myd['measure']

    myl = list(uas_df.loc[i+186])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    print(bucket_filepath)

In [ ]:
uas_df = df[df['measure'] == 'uas']
uas_lines = lines[187:218]

for i in range(0,len(uas_df.index)):
    words = uas_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(uas_df.loc[i+186])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(uas_df.loc[i+186])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    print(bucket_filepath)
#     s3_push_delete_local(local_file, bucket, bucket_filepath)/

In [ ]:
vas_df = df[df['measure'] == 'vas']
vas_lines = lines[218:249]

for i in range(0,len(vas_df.index)):
    words = vas_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(vas_df.loc[i+217])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(vas_df.loc[i+217])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
# THE CODE BELOW APPLIES TO INPUTS THAT HAD TO BE REDOWNLOADED DUE TO CORRUPT FILES (HOWEVER THAT HAPPENED IDK)
# ONCE BELOW IS RUN, ASSUME THAT THE CODE ABOVE IS FINAL

In [ ]:
files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
files_2006=grepfxn('2006',outs)
files_2046=grepfxn('2046',outs)
files_2051=grepfxn('2051',outs)
files_2056=grepfxn('2056',outs)
files_2061=grepfxn('2061',outs)
files_2066=grepfxn('2066',outs)
files_2071=grepfxn('2071',outs)
files_2076=grepfxn('2076',outs)
files_2081=grepfxn('2081',outs)
files_2086=grepfxn('2086',outs)
files_2091=grepfxn('2091',outs)
files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_1950

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# for the redownloaded files (e.g. 1950-1954)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

pr_df = df[df['measure'] == 'pr'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
pr_lines = lines[1] # 1 to 31 are the rows associated with precipitation netCDF files

for i in range(0,len(pr_df.index)):
    words = pr_lines[i].split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(pr_df.loc[i])
    myl = list(pr_df.loc[i]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
        
    to_dir= home + '/junkbox/' + myd['measure']
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd

    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)


In [9]:
# print(outs)

# files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
# files_2006=grepfxn('2006',outs)
# files_2046=grepfxn('2046',outs)
# files_2051=grepfxn('2051',outs)
# files_2056=grepfxn('2056',outs)
# files_2061=grepfxn('2061',outs)
# files_2066=grepfxn('2066',outs)
# files_2071=grepfxn('2071',outs)
# files_2076=grepfxn('2076',outs)
# files_2081=grepfxn('2081',outs)
# files_2086=grepfxn('2086',outs)
# files_2091=grepfxn('2091',outs)
# files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_1965

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

home = os.getenv('HOME')
wget = 'wget -nc -c -nd ' 

# print(df)
# print(lines)

# for the redownloaded files (e.g. 1950-1954)

vas_df = df[df['measure'] == 'vas']
print(vas_df)

vas_lines = lines[218:249]
# print(vas_lines)
vas_lines2 = vas_lines[3]
print(vas_lines2)

         dataset measure   model      r6       epoch start_year end_year  \
7  macav2metdata     vas  MIROC5  r1i1p1  historical       1965     1969   

  region frequency  
7  CONUS     daily  
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc" -O macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc  



In [34]:

for i in range(0,len(vas_df.index)):
    words = vas_lines2.split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    
    myd = dict(vas_df.loc[7])
    
    to_dir= home + '/junkbox/' + myd['measure']
    
    try:
        os.mkdir(to_dir);
    except:
        pass
    
    myl = list(vas_df.loc[7])
    
    crafted_file_name = '_'.join(myl) + '.nc'
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    
    wget_cmd2 = wget + wget_cmd
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)


"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
dev-et-data raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc


In [33]:
# vas_lines[0].split() 
vas_lines2.split() 

['wget',
 '-nc',
 '-c',
 '-nd',
 '"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc"',
 '-O',
 'macav2metdata_vas_MIROC5_r1i1p1_historical_1950_1954_CONUS_daily.nc']

In [35]:
print(vas_lines2)
print(words)
print(http_line)
print(vas_lines2)
# print(wget)
print(http_root)
print(wget_cmd)
# print(wget_cmd2)

wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc" -O macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc  

['wget', '-nc', '-c', '-nd', '"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"', '-O', 'macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc']
"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc"
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc" -O macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc  

http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_196

In [37]:
print(local_file)
print(bucket)
print(bucket_filepath)

/home/jupyter-rouze/junkbox/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc
dev-et-data
raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc


In [36]:
my_file= 'dev-et-data/raw/vas/macav2metdata_vas_MIROC5_r1i1p1_historical_1965_1969_CONUS_daily.nc'
import fsspec
s3path = my_file
#fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
fs = fsspec.filesystem('s3')
print(fs)
fobj = fs.open(s3path)
import xarray as xr
ds = xr.open_dataset(fobj)
print(ds)

<xarray.Dataset>
Dimensions:         (crs: 1, lat: 585, lon: 1386, time: 1826)
Coordinates:
  * lat             (lat) float64 25.06 25.1 25.15 25.19 ... 49.31 49.35 49.4
  * lon             (lon) float64 235.2 235.3 235.3 235.4 ... 292.9 292.9 292.9
  * time            (time) datetime64[ns] 1965-01-01 1965-01-02 ... 1969-12-31
  * crs             (crs) int32 1
Data variables:
    northward_wind  (time, lat, lon) float32 ...
Attributes:
    description:                     Multivariate Adaptive Constructed Analog...
    id:                              MACAv2-METDATA
    naming_authority:                edu.uidaho.reacch
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    Metadata_Link:                   Empty(dtype=dtype('S1'))
    cdm_data_type:                   GRID
    title:                           Downscaled daily meteorological data of ...
    summary:                         This archive contains daily downscaled m...
    keywords:                        d

In [46]:
files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
files_2006=grepfxn('2006',outs)
files_2046=grepfxn('2046',outs)
files_2051=grepfxn('2051',outs)
files_2056=grepfxn('2056',outs)
files_2061=grepfxn('2061',outs)
files_2066=grepfxn('2066',outs)
files_2071=grepfxn('2071',outs)
files_2076=grepfxn('2076',outs)
files_2081=grepfxn('2081',outs)
files_2086=grepfxn('2086',outs)
files_2091=grepfxn('2091',outs)
files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_2006

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# for the redownloaded files (e.g. 1950-1954)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

print(df)

pr_df = df[df['measure'] == 'pr'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
pr_lines = lines[13] # 1 to 31 are the rows associated with precipitation netCDF files

# print(lines)
print(pr_lines)
print(pr_lines.split())

         dataset measure   model      r6  epoch start_year end_year region  \
0  macav2metdata      pr  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
1  macav2metdata  rhsmax  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
2  macav2metdata  rhsmin  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
3  macav2metdata    rsds  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
4  macav2metdata  tasmin  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
5  macav2metdata  tasmax  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
6  macav2metdata     uas  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   
7  macav2metdata     vas  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   

  frequency  
0     daily  
1     daily  
2     daily  
3     daily  
4     daily  
5     daily  
6     daily  
7     daily  
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc" -O macav2metdat

In [47]:

for i in range(0,len(pr_df.index)):
    words = pr_lines.split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(pr_df.loc[i])
    myl = list(pr_df.loc[i]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
        
    to_dir= home + '/junkbox/' + myd['measure']
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd

    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)


"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc"
dev-et-data raw/pr/macav2metdata_pr_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc


In [54]:
files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
files_2006=grepfxn('2006',outs)
files_2046=grepfxn('2046',outs)
files_2051=grepfxn('2051',outs)
files_2056=grepfxn('2056',outs)
files_2061=grepfxn('2061',outs)
files_2066=grepfxn('2066',outs)
files_2071=grepfxn('2071',outs)
files_2076=grepfxn('2076',outs)
files_2081=grepfxn('2081',outs)
files_2086=grepfxn('2086',outs)
files_2091=grepfxn('2091',outs)
files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_2006

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# for the redownloaded files (e.g. 1950-1954)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

# print(df)

rsds_df = df[df['measure'] == 'rsds'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
rsds_lines = lines[106] # 1 to 31 are the rows associated with precipitation netCDF files

# print(lines)
print(rsds_df)
print(rsds_lines)
print(rsds_lines.split())

         dataset measure   model      r6  epoch start_year end_year region  \
3  macav2metdata    rsds  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   

  frequency  
3     daily  
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc" -O macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc  

['wget', '-nc', '-c', '-nd', '"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc"', '-O', 'macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc']


In [56]:

for i in range(0,len(rsds_df.index)):
    words = rsds_lines.split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(rsds_df.loc[3])
    myl = list(rsds_df.loc[3]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
        
    to_dir= home + '/junkbox/' + myd['measure']
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd

    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)


"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc


In [61]:
files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
files_2006=grepfxn('2006',outs)
files_2046=grepfxn('2046',outs)
files_2051=grepfxn('2051',outs)
files_2056=grepfxn('2056',outs)
files_2061=grepfxn('2061',outs)
files_2066=grepfxn('2066',outs)
files_2071=grepfxn('2071',outs)
files_2076=grepfxn('2076',outs)
files_2081=grepfxn('2081',outs)
files_2086=grepfxn('2086',outs)
files_2091=grepfxn('2091',outs)
files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_2006

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# for the redownloaded files (e.g. 1950-1954)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

# print(df)

tasmin_df = df[df['measure'] == 'tasmin'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
tasmin_lines = lines[137] # 1 to 31 are the rows associated with precipitation netCDF files

# print(lines)
print(tasmin_df)
print(tasmin_lines)
print(tasmin_lines.split())

         dataset measure   model      r6  epoch start_year end_year region  \
4  macav2metdata  tasmin  MIROC5  r1i1p1  rcp45       2006     2010  CONUS   

  frequency  
4     daily  
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc" -O macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc  

['wget', '-nc', '-c', '-nd', '"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc"', '-O', 'macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc']


In [62]:

for i in range(0,len(tasmin_df.index)):
    words = tasmin_lines.split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(tasmin_df.loc[4])
    myl = list(tasmin_df.loc[4]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
        
    to_dir= home + '/junkbox/' + myd['measure']
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd

    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc"
dev-et-data raw/tasmin/macav2metdata_tasmin_MIROC5_r1i1p1_rcp45_2006_2010_CONUS_daily.nc


In [66]:
files_1950=grepfxn('1950',outs)
files_1965=grepfxn('1965',outs)
files_2006=grepfxn('2006',outs)
files_2046=grepfxn('2046',outs)
files_2051=grepfxn('2051',outs)
files_2056=grepfxn('2056',outs)
files_2061=grepfxn('2061',outs)
files_2066=grepfxn('2066',outs)
files_2071=grepfxn('2071',outs)
files_2076=grepfxn('2076',outs)
files_2081=grepfxn('2081',outs)
files_2086=grepfxn('2086',outs)
files_2091=grepfxn('2091',outs)
files_2096=grepfxn('2096',outs)

# files_redownload = files_1950
files_redownload = files_2061

# THIS IS FOR THE REDOWNLOADED FILES THAT WERE PROBABLY CORRUPT IN SAME MANNER

dfis=[]
for out in files_redownload:
    a=out.split('_')
    dfi = {'dataset':a[0], 'measure':a[1], 'model':a[2], 'r6':a[3], 'epoch':a[4], 'start_year':a[5], 'end_year':a[6],
          'region':a[7], 'frequency':a[8]}
    dfis.append(dfi)

# for the redownloaded files (e.g. 1950-1954)

# Create a dataframe consisting of metdata info on each of the netCDFs 
df = pd.DataFrame(dfis)

# print(df)

rsds_df = df[df['measure'] == 'rsds'] # extract all of the rows that have 'pr' or precipitation within the netcdf file name
rsds_lines = lines[117] # 1 to 31 are the rows associated with precipitation netCDF files

# print(lines)
print(rsds_df)
print(rsds_lines)
print(rsds_lines.split())

         dataset measure   model      r6  epoch start_year end_year region  \
3  macav2metdata    rsds  MIROC5  r1i1p1  rcp45       2061     2065  CONUS   

  frequency  
3     daily  
wget -nc -c -nd "http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc" -O macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc  

['wget', '-nc', '-c', '-nd', '"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc"', '-O', 'macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc']


In [67]:

for i in range(0,len(rsds_df.index)):
    words = rsds_lines.split()
    for w in words:
        if 'http' in w:
            print(w)
            http_line = w

    #     http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    #     myd = dict(pr_df.loc[i])
    #     myl = list(pr_df.loc[i]) # get each precipitation row
    #     crafted_file_name = '_'.join(myl) + '.nc'
    #     wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    #     wget_cmd2 = wget + wget_cmd

    myd = dict(rsds_df.loc[3])
    myl = list(rsds_df.loc[3]) # get each precipitation row
    http_root=http_line.split('macav2metdata/')[0][1:len(http_line)-1]
    crafted_file_name = '_'.join(myl) + '.nc'
    wget = 'wget -nc -c -nd ' 
        
    to_dir= home + '/junkbox/' + myd['measure']
    
    wget_cmd = http_root +  ' -O ' + to_dir + '/' + crafted_file_name 
    wget_cmd2 = wget + wget_cmd

    try:
        os.mkdir(to_dir);
    except:
        pass
    
    os.system(wget_cmd2)
    
    bucket = 'dev-et-data'
    
    local_file = to_dir + '/' + crafted_file_name
    
    # output filepath moving from the mini-Pangeo to the Amazon s3 databucket
    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)


"http://thredds.northwestknowledge.net:8080/thredds/fileServer/MACAV2/MIROC5/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc"
dev-et-data raw/rsds/macav2metdata_rsds_MIROC5_r1i1p1_rcp45_2061_2065_CONUS_daily.nc


# Move it to the cloud 

In [ ]:
import boto3
def s3_push_delete_local(local_file, bucket, bucket_filepath):
        """
        This function will move the model outputs from a local folder to a cloud bucket.
        :param local_file: path the the local geo file
        :param outpath: path of a directory to be created in the cloud bucket
        :param bucket: name of the cloud bucket = 'dev-et-data'
        :param bucket_folder: "folder" in cloud bucket  = 'v1DRB_outputs'
        :return:
        """

        s3 = boto3.client('s3')
        with open(local_file, "rb") as f:
            print(bucket, bucket_filepath)
            s3.upload_fileobj(f, bucket, bucket_filepath)
        os.remove(local_file)


In [ ]:
bucket = 'dev-et-data'

local_file = to_dir + '/' + crafted_file_name

bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name

print(local_file)
print(bucket)
print(bucket_filepath)


In [ ]:
%%time
s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
len(pr_df.index)

In [ ]:
pr_df = df[df['measure'] == 'pr']

for i in range(0,len(pr_df.index)):
    
    myl = list(pr_df.loc[i])

    crafted_file_name = '_'.join(myl) + '.nc'
    
    bucket = 'dev-et-data'

    local_file = to_dir + '/' + crafted_file_name

    bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
    
    s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
import fsspec


In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)

In [ ]:
file_objects = fs.ls('dev-et-data')
file_objects

In [ ]:
file_objects = fs.ls('dev-et-data/raw/pr/')
file_objects

In [ ]:
next_item = pr_df.loc[pr_df['start_year'] =='1970']

In [ ]:
next_item.values

In [ ]:
myl=list(next_item.values[0])

In [ ]:
myl

In [ ]:
crafted_file_name = '_'.join(myl) + '.nc'
crafted_file_name

In [ ]:
wget_cmd = http_root + '/' + from_dir + '/' + crafted_file_name + ' -O ' + to_dir + '/' + crafted_file_name 
wget_cmd = wget + wget_cmd
wget_cmd


In [ ]:
%%time
os.system(wget_cmd)

In [ ]:
local_file = to_dir + '/' + crafted_file_name
bucket_filepath = 'raw/' + myd['measure'] + '/' + crafted_file_name
bucket_filepath

In [ ]:
%%time
s3_push_delete_local(local_file, bucket, bucket_filepath)

In [ ]:
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
file_objects = fs.ls('dev-et-data/raw/pr/')
file_objects

In [ ]:
!aws s3 ls dev-et-data/raw/pr/